In [ ]:
! conda install -y nodejs
! pip3 install dask_labextension
! jupyter labextension install dask-labextension
! sudo apt-get update
! sudo apt-get -y install build-essential
! pip3 install bayesian_optimization
! pip3 install cython
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import cython
import time
import pandas as pd
%load_ext Cython

In [55]:
train_values_short = pd.read_csv('train_values_short1.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [6]:
%%cython
def hassanat1(double[:] x, double[:] y, double[:] factores):
    cdef int n = x.shape[0]
    cdef double res = 0
    cdef double d, minimo, maximo

    for i in range(n):
        minimo = min(x[i],y[i])
        maximo = max(x[i],y[i])
        if (minimo >=0):
            d =  1 - ( (1+minimo)/(1+maximo) )
        else:
            d =  1 - ( (1+minimo+abs(minimo))/(1+maximo+abs(minimo)) )
        res += d * factores[i]
    
    return res

In [3]:
factores = np.array([1.82264756, 0.14128378, 0.27999595, 0.4755266 , 2.30883141,
       1.02996122, 1.07966807, 1.10292129, 0.83811159, 0.80736305,
       0.74243681, 4.93845144, 4.93845144, 7.76639614, 0.03964061550504756, 0.09684727174529291, 0.025805063269526796])\
        .astype('double')
ids = train_values_short.index

In [110]:
lista = []
for i in range(10,20):
    x_train =train_values_short.copy(deep=True)
    y_train = train_labels.copy(deep=True)
    x_train.drop(index=ids[i], inplace=True  )
    y_train.drop(index=ids[i], inplace=True )
    knn = KNeighborsClassifier(weights="distance",n_neighbors=22,
                  metric=hassanat1, metric_params={'factores':factores})
    knn.fit(x_train, y_train.values.ravel())
    lista.append( knn.predict_proba([train_values_short.loc[ids[i] ]])[0])

In [112]:
df = pd.DataFrame(lista, index=ids[10:20], columns=['proba_grado1', 'proba_grado2', 'proba_grado3'])

In [114]:
df.head()

,proba_grado1,proba_grado2,proba_grado3
building_id,,,
7962,0.000000,0.067170,0.932830
452227,0.120694,0.637787,0.241519
444381,0.000000,1.000000,0.000000
287845,0.000000,0.000000,1.000000
605134,0.000000,0.766557,0.233443


In [ ]:
df.to_csv("probas.csv",index=True)